# Webscraper energieprijzen
URLs:
* https://www.epexspot.com/en/market-data
* https://www.energyzero.nl/energiecontract/stroom-en-gas/flextarieven

In [63]:
import pandas as pd
import requests
import webbrowser
from pathlib import Path
from datetime import date, timedelta, datetime
import re
import lxml.html as html
import dropbox

In [74]:
# We willen de prijzen van morgen
today = date.today()
tomorrow = today + timedelta(1)

In [79]:
# Browse naar website
url = "https://www.epexspot.com/en/market-data"
params = {
    "market_area": "NL",
    "trading_date": today,
    "delivery_date": tomorrow,
    "modality": "Auction",
    "sub_modality": "DayAhead",
    "product": 60,
    "data_mode": "table",
    
}
print("GET", params)
page = requests.get(url, params=params)
assert page.status_code == 200

# Get html content
content = html.fromstring(page.content)

GET {'market_area': 'NL', 'trading_date': datetime.date(2021, 4, 6), 'delivery_date': datetime.date(2021, 4, 7), 'modality': 'Auction', 'sub_modality': 'DayAhead', 'product': 60, 'data_mode': 'table'}


In [85]:
# Let op: Bij een toekomstige datum waar de data nog niet van bekend is, springt de URL terug naar die van vandaag
# Dus check de datum in <h2>
h2 = content.cssselect("h2")
assert len(h2) == 1
h2 = h2[0].text

# Alle aansluitende whitespace reduceren naar 1 spatie en strip uiteinden
h2 = re.sub(r"\s+", " ", h2).strip()
print("<h2>", h2)

# Pak laatste element na >
m = re.match(r".* > (.*)", h2)
d = datetime.strptime(m.group(1), "%d %B %Y").date()
assert d == tomorrow

<h2> Auction > Day-Ahead > 60min > NL > 07 April 2021


In [84]:
# Print ook het laatste moment van updaten
last_update = re.sub(r"\s+", " ", content.cssselect("span.last-update")[0].text)
print(last_update)

Last update: 06 April 2021 (12:43:53 CET/CEST) 


In [28]:
# Selecteer de <table>
tbody = content.cssselect("tbody")
assert len(tbody) == 1
tbody = tbody[0]
tr_s = tbody.cssselect("tr")
assert len(tr_s) == 24

In [12]:
# Optioneel: bekijk html in browser
path = Path().resolve()
filename = "tmp.html"
with open(path / filename, "w") as f:
    f.write(str(page.content))

webbrowser.open_new_tab(f"file:///{path / filename}")

True

In [24]:
# Scrape de tabel
d = {
    "day": tomorrow,
    "hours": sorted(range(24)),
    "buy_volume": [],
    "sell_volume": [],
    "volume": [],
    "price": [],
}
column_headers = ["buy_volume", "sell_volume", "volume", "price"]
for tr in tr_s:
    for column_header, td in zip(column_headers, tr.cssselect("td")):
        d[column_header].append(td.text)

df = pd.DataFrame(d)
df

,day,hours,buy_volume,sell_volume,volume,price
0,2021-04-08,0,"2,281.5","2,670.2","2,670.2",54.90
1,2021-04-08,1,"2,198.2","3,032.2","3,032.2",45.62
2,2021-04-08,2,"2,147.1","3,321.1","3,321.1",45.40
3,2021-04-08,3,"2,212.6","3,464.1","3,464.1",40.67
4,2021-04-08,4,"2,220.2","3,521.9","3,521.9",39.52
5,2021-04-08,5,"2,262.6","3,354.7","3,354.7",42.60
6,2021-04-08,6,"2,402.8","3,236.9","3,236.9",57.34
7,2021-04-08,7,"2,467.9","2,985.3","2,985.3",77.88
8,2021-04-08,8,"2,847.7","2,922.8","2,922.8",82.47
9,2021-04-08,9,"2,739.1","3,226.2","3,226.2",65.12


In [ ]:
# Connect to Dropbox
access_token = "W9SIGAIOPbcAAAAAAAAAAcbg_ogFc8xw3lBhJeybL7phj6AZdrV5_CyQCgnPFZlt"
dbx = dropbox.Dropbox(access_token)
assert dbx.users_get_current_account().name.abbreviated_name == "AB"

In [19]:
# Upload csv
binary_csv = df.to_csv(index=False).encode('utf-8')
filename = f"/epexspot_{tomorrow}.csv"
dbx.files_upload(binary_csv, filename, mode=dropbox.files.WriteMode("overwrite"))

FileMetadata(client_modified=datetime.datetime(2021, 4, 6, 12, 16, 55), content_hash='8afd5f42714e607efe47e712ad96f2888e8e633e638abc3f8b9cb03bae6be17d', export_info=NOT_SET, file_lock_info=NOT_SET, has_explicit_shared_members=NOT_SET, id='id:jwa9lCFnHysAAAAAAAAKuA', is_downloadable=True, media_info=NOT_SET, name='epexspot_2021-04-07.csv', parent_shared_folder_id=NOT_SET, path_display='/epexspot_2021-04-07.csv', path_lower='/epexspot_2021-04-07.csv', property_groups=NOT_SET, rev='015bf4cca4d1ca6000000022fc32290', server_modified=datetime.datetime(2021, 4, 6, 12, 16, 56), sharing_info=NOT_SET, size=1236, symlink_info=NOT_SET)

In [20]:
# Print files in folder
result = dbx.files_list_folder('')
for entry in result.entries:
    print(entry.path_display)

/epexspot_2021-04-06.csv
/epexspot_2021-04-05.csv
/epexspot_2021-04-07.csv
